# Few-shot Learning 

In [1]:
%xmode minimal

import os
import json

# Turn off logging for TF
import logging
# tf.get_logger().setLevel(logging.ERROR)
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  # disable GPU devices
os.environ["TFDS_DATA_DIR"] = os.path.expanduser("~/tensorflow_datasets")  # default location of tfds database

import os
# os.environ["KERAS_BACKEND"] = "tensorflow"
# os.environ["KERAS_BACKEND"] = "jax"
os.environ["KERAS_BACKEND"] = "torch"

import keras
from keras import layers, models, ops

import tensorflow as tf
import tensorflow_datasets as tfds

import librosa
import librosa.display

import numpy as np
from matplotlib import pyplot as plt

from pathlib import Path

# from IPython.display import Audio

# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

Exception reporting mode: Minimal


In [2]:
import dpmhm
# dpmhm.datasets.get_dataset_list()

from dpmhm.datasets import preprocessing, feature, utils, transformer, query_parameters

In [3]:
outdir = Path('/home/han/tmp/dpmhm/few-shot')
os.makedirs(outdir, exist_ok=True)

## Load a dataset

In [4]:
_func = lambda x, sr: feature.spectral_features(
    x, sr, 'spectrogram',
    # n_mfcc=256,
    time_window=0.025, hop_step=0.0125,
    # n_fft=512,
    normalize=False, to_db=True)[0]

compactor_kwargs = dict(
    channels=[],
    keys=['FaultLocation', 'FaultComponent', 'FaultSize'],
)

window_kwargs = dict(
    window_size=(64,64), 
    hop_size=(64,64),
    # hop_size=(32,32)    
)

ds0, full_label_dict = dpmhm.datasets.spectral_window_pipeline(
    'CWRU', 
    spectral_feature=_func,
    compactor_kwargs=compactor_kwargs,
    window_kwargs=window_kwargs,
)

print('Full dictionary of labels:', full_label_dict)

label_index_dict = preprocessing.get_label_mapping(list(full_label_dict.keys()))
print('Integer index of labels:', label_index_dict)

Full dictionary of labels: {'7f67e45381c3652c': ['FanEnd', 'OuterRace3', '0.014'], '8b9e80c02e1fca5b': ['FanEnd', 'OuterRace6', '0.007'], 'd766ecd2592ce5ec': ['DriveEnd', 'Ball', '0.021'], 'e27e22f1f5037a20': ['DriveEnd', 'InnerRace', '0.014'], '8af14bb8ad669337': ['FanEnd', 'OuterRace12', '0.007'], '2533c59036dfe8c8': ['FanEnd', 'InnerRace', '0.007'], 'd6765bfdf1aca38f': ['DriveEnd', 'Ball', '0.007'], 'a760eef52ceaa6f9': ['FanEnd', 'OuterRace3', '0.021'], 'b0a92d9d7379d8ce': ['DriveEnd', 'InnerRace', '0.007'], '1c80dbfc87966d6e': ['FanEnd', 'InnerRace', '0.014'], 'd8957867a1fc0519': ['DriveEnd', 'OuterRace3', '0.007'], 'd45bbeb3b8a72222': ['DriveEnd', 'OuterRace6', '0.007'], '55503c950ed81973': ['FanEnd', 'Ball', '0.014'], '60836667e7ee1dec': ['FanEnd', 'OuterRace3', '0.007'], '605222dceca4b27e': ['FanEnd', 'Ball', '0.021'], '6c2ba36f712d55e4': ['DriveEnd', 'OuterRace6', '0.014'], 'dc8bcb86c369e78b': ['FanEnd', 'InnerRace', '0.021'], '9c54396620a4b6a3': ['DriveEnd', 'OuterRace6', '0.0

### Split for few-shot learning with OOD

In [ ]:
preproc = preprocessing.get_mapping_supervised(list(full_label_dict.keys()))

ds1 = utils.restore_shape(
    ds0.map(preproc, num_parallel_calls=tf.data.AUTOTUNE),
    key=0
)

In [ ]:
# OOD labels
labels_ood = [1, 2]
labels = [l for l in label_index_dict.values() if l not in labels_ood]

# Extract OOD samples and concatenate into a single OOD category
# use key=1 here because `ds1` is a tuple dataset and 1 is the index of the label field
foo = utils.extract_by_category(ds1, labels_ood, key=1)  
ds_ood = None
for k, dv in foo.items():
    try:
        ds_ood = ds_ood.concatenate(dv)
    except:
        ds_ood = dv

# Few-shot split
splits = {'train':0.2, 'val':0.7, 'test':0.1}
ds_split = utils.split_dataset(ds1, splits=splits, labels=labels, key=1)

#### Export and reload the preprocessed dataset 

For a better performance

In [12]:
for k, dv in ds_split.items():
    dv.save(str(outdir/k))

ds_ood.save(str(outdir/'ood'))

In [ ]:
ds_split = {}

for k in ['train', 'val', 'test']:
    ds_split[k] = tf.data.Dataset.load(str(outdir/k))

ds_ood = tf.data.Dataset.load(str(outdir/'ood'))

print(ds_ood.element_spec, ds_ood.cardinality())

# ds_split['train'].cardinality()

## Train a VGGish network

In [12]:
batch_size = 64

ds_train = ds_split['train']\
    .shuffle(1000, reshuffle_each_iteration=True)\
    .batch(batch_size, drop_remainder=True)\
    .prefetch(tf.data.AUTOTUNE)
ds_val = ds_split['val'].batch(batch_size, drop_remainder=True)
ds_test = ds_split['test'].batch(batch_size, drop_remainder=True)

ds_ood_test = ds_ood.batch(batch_size, drop_remainder=True)

n_classes = len(full_label_dict) + 1

input_shape = ds_split['train'].element_spec[0].shape

In [13]:
from keras.applications import VGG16

base_model = VGG16(include_top=False, weights='imagenet', input_shape=input_shape, classes=n_classes)

base_model.trainable = False

In [14]:
x = layers.Input(input_shape)

adapt_model = models.Sequential([
    layers.Flatten(name="flatten"),
    layers.Dense(4096, activation="relu", name="fc1"),
    layers.Dense(4096, activation="relu", name="fc2"),
    layers.Dense(n_classes, activation=None, name="predictions")
])

y = adapt_model(base_model(x))

# Equivalent:
# x = base_model(x)
# x = layers.Flatten(name="flatten")(x)
# x = layers.Dense(4096, activation="relu", name="fc1")(x)
# x = layers.Dense(4096, activation="relu", name="fc2")(x)
# y = layers.Dense(n_classes, activation=None, name="predictions")(x)

In [15]:
model = models.Model(x, y)

from_logits = 'softmax' not in str(model.layers[-1].get_layer('predictions').activation)

model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=from_logits),
    metrics=['accuracy'],
)

In [17]:
history = model.fit(
    ds_train,
    validation_data=ds_val,
    epochs=10,
    callbacks=tf.keras.callbacks.EarlyStopping(verbose=1, patience=3),
)

Epoch 1/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 22s 2s/step - accuracy: 0.7479 - loss: 0.8147 - val_accuracy: 0.8207 - val_loss: 0.5760
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - accuracy: 0.8765 - loss: 0.3828 - val_accuracy: 0.8821 - val_loss: 0.3911
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - accuracy: 0.9301 - loss: 0.2492 - val_accuracy: 0.8798 - val_loss: 0.4583
Epoch 4/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.9578 - loss: 0.1732 - val_accuracy: 0.8913 - val_loss: 0.3275
Epoch 5/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.9612 - loss: 0.1537 - val_accuracy: 0.8961 - val_loss: 0.3444
Epoch 6/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - accuracy: 0.9758 - loss: 0.1078 - val_accuracy: 0.8957 - val_loss: 0.3493
Epoch 7/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 22s 2s/step - accuracy: 0.9723 - loss: 0.1212 - val_accuracy: 0.9198 - val_loss: 0.2992
Epoch 8/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.9893 - loss: 0.0516 - val_accuracy: 0.9395 - val_loss:

Trained model has a descent performance on test data:

In [18]:
model.evaluate(ds_test)

7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 357ms/step - accuracy: 0.9434 - loss: 0.2846


[0.22488360106945038, 0.9419642686843872]

On the contrary, on OOD data completely failed. 

In [20]:
model.evaluate(ds_ood_test)

6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 317ms/step - accuracy: 0.0000e+00 - loss: 24.1354


[25.666162490844727, 0.0]

### Fine tuning

Fine tuning in the few-shot learning scenario doesn't really improve the model's performance.

In [21]:
base_model.trainable = True

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=from_logits),
    metrics=['accuracy'],
)

In [22]:
history = model.fit(
    ds_train,
    validation_data=ds_val,
    epochs=2,
    callbacks=tf.keras.callbacks.EarlyStopping(verbose=1, patience=3),
)

Epoch 1/2
13/13 ━━━━━━━━━━━━━━━━━━━━ 31s 2s/step - accuracy: 0.9977 - loss: 0.0149 - val_accuracy: 0.9321 - val_loss: 0.3310
Epoch 2/2
13/13 ━━━━━━━━━━━━━━━━━━━━ 31s 2s/step - accuracy: 0.9939 - loss: 0.0200 - val_accuracy: 0.9457 - val_loss: 0.2088


In [24]:
model.evaluate(ds_test)

7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 336ms/step - accuracy: 0.9438 - loss: 0.3563


[0.261015921831131, 0.9486607313156128]

In [25]:
model.evaluate(ds_ood_test)

6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 339ms/step - accuracy: 0.0000e+00 - loss: 26.4604


[28.41253089904785, 0.0]

### Adaptation on OOD data

We may adapt the model on OOD data, however this may incur catastrophic forget.

In [36]:
# Split the OOD data
ds_ood_split = utils.split_dataset(ds_ood, {'train':0.2, 'val':0.7, 'test':0.1}, key=1)

batch_size = 16

ds_ood_train = ds_ood_split['train']\
    .shuffle(1000, reshuffle_each_iteration=True)\
    .batch(batch_size, drop_remainder=True)\
    .prefetch(tf.data.AUTOTUNE)
ds_ood_val = ds_ood_split['val'].batch(batch_size, drop_remainder=True)
ds_ood_test = ds_ood_split['test'].batch(batch_size, drop_remainder=True)

In [44]:
base_model.trainable = True

model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=from_logits),
    metrics=['accuracy'],
)

In [45]:
history = model.fit(
    ds_ood_train,
    validation_data=ds_ood_val,
    epochs=10,
    callbacks=tf.keras.callbacks.EarlyStopping(verbose=1, patience=3),
)

Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - accuracy: 0.0000e+00 - loss: 27.8739 - val_accuracy: 0.0000e+00 - val_loss: 3.5991
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - accuracy: 0.2042 - loss: 2.8744 - val_accuracy: 0.4805 - val_loss: 2.8254
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - accuracy: 0.4021 - loss: 2.1012 - val_accuracy: 0.5234 - val_loss: 0.7230
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - accuracy: 0.5021 - loss: 0.8861 - val_accuracy: 0.4961 - val_loss: 0.7041
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - accuracy: 0.3688 - loss: 0.7533 - val_accuracy: 0.5000 - val_loss: 0.6899
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - accuracy: 0.5500 - loss: 0.7189 - val_accuracy: 0.4922 - val_loss: 0.6937
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - accuracy: 0.6542 - loss: 0.6928 - val_accuracy: 0.5000 - val_loss: 0.7093
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - accuracy: 0.5354 - loss: 0.7058 - val_accuracy: 0.5195 - val_loss: 0.7011
Epoch 8

In [47]:
model.evaluate(ds_ood_test)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step - accuracy: 0.5625 - loss: 0.6792


[0.6783491373062134, 0.5625]

In [46]:
model.evaluate(ds_test)

7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 346ms/step - accuracy: 0.0000e+00 - loss: 11.6353


[11.451746940612793, 0.0]

## Alternative of the splitting scheme

The preprocessing step which converts string labels to integer index can be applied after the split, as shown below.

In [ ]:
# Split for few-shot learning with OOD

# OOD labels
# labels_ood = ['0881fa248109963a']  # CWRU: the 3 channels data contain no normal samples
labels_ood = ['55503c950ed81973']  # ['FanEnd', 'Ball', '0.014']
# Other labels
labels = [l for l in full_label_dict if l not in labels_ood]
# labels = list(full_label_dict.keys())

# Extract OOD samples and concatenate into a single OOD category
foo = utils.extract_by_category(ds0, labels_ood)
for k, dv in foo.items():
    try:
        ds_ood = ds_ood.concatenate(dv)
    except:
        ds_ood = dv

# Few-shot split
splits = {'train':0.2, 'val':0.7, 'test':0.1}
ds_split = utils.split_dataset(ds0, splits=splits, labels=labels)

In [ ]:
preproc = preprocessing.get_mapping_supervised(list(full_label_dict.keys()))

batch_size = 64

ds_train = ds_split['train']\
    .map(preproc, num_parallel_calls=tf.data.AUTOTUNE)\
    .shuffle(1000, reshuffle_each_iteration=True)\
    .batch(batch_size, drop_remainder=True)\
    .prefetch(tf.data.AUTOTUNE)
ds_val = ds_split['val']\
    .map(preproc, num_parallel_calls=tf.data.AUTOTUNE)\
    .batch(batch_size, drop_remainder=True)
ds_test = ds_split['test']\
    .map(preproc, num_parallel_calls=tf.data.AUTOTUNE)\
    .batch(batch_size, drop_remainder=True)

In [ ]:
with open(outdir/'full_labels.json', 'w') as fp:
    json.dump(full_label_dict,fp)

with open(outdir/'label_mapping.json', 'w') as fp:
    json.dump(label_index_dict,fp)

# dp_split = {}
# for k, dv in ds_split.items():
#     dp_split[k] = utils.restore_shape(
#         ds_split[k].map(preproc, num_parallel_calls=tf.data.AUTOTUNE)
#     )
# )
# # ds_size = utils.get_dataset_size(ds_window)

# EOF